<a href="https://colab.research.google.com/github/geopan2000/Loan-Approval-Prediction/blob/main/LoanApprovalPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Loan-Approval-Prediction.git

# Change directory to the cloned repository
%cd Loan-Approval-Prediction

Cloning into 'Loan-Approval-Prediction'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 49 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 2.12 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction


In [63]:
import pandas as pd
import numpy as np
!pip install xgboost
import xgboost as xgb

!pip install imbalanced-learn
!pip install scikit-optimize
!pip install shap
from skopt import BayesSearchCV
from skopt.space import Real, Integer

from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from google.colab import files

In [64]:
train_set = pd.read_csv('/content/Loan-Approval-Prediction/data/train.csv') # Load train set
test_set = pd.read_csv('/content/Loan-Approval-Prediction/data/test.csv') #Load test set

In [65]:
print(train_set.isna().sum()) #inspect train set for possible missing values
print(test_set.isna().sum()) #inspect test set for possible missing values

id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64
id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64


In [66]:
print(train_set.dtypes) #inspect data types

id                              int64
person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
loan_status                     int64
dtype: object


In [67]:
print('The size of the train set is:', train_set.shape)
print('The size of the test set is:', test_set.shape) # the target variable is missing from the set

The size of the train set is: (58645, 13)
The size of the test set is: (39098, 12)


**Inspect which features are object type and what are the differnet values inside them**

In [68]:
print(train_set.info())
print(train_set['person_home_ownership'].value_counts())
print(train_set['loan_intent'].value_counts())
print(train_set['loan_grade'].value_counts())
print(train_set['cb_person_default_on_file'].value_counts())
print(train_set['loan_status'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

**Feature Engineering**

In [69]:
# Ensure to map and assign the results back to the columns
train_set['cb_person_default_on_file'] = train_set['cb_person_default_on_file'].map({'Y': 1, 'N': 0}).astype(np.float32)

# Keep a copy of the original loan_grade column before encoding
train_set['loan_grade_0'] = train_set['loan_grade']

train_set['loan_grade'] = train_set['loan_grade'].map({'A': 0.1, 'B': 0.2, 'C': 0.4, 'D': 0.5, 'E': 0.6, 'F': 0.8, 'G': 1}).astype(np.float32)

# Dummy encoding for categorical columns
train_set = pd.get_dummies(train_set, columns=['loan_grade_0', 'person_home_ownership', 'loan_intent'])

# Drop the 'id' column as it's not needed
train_set.drop(['id'], axis=1, inplace=True)

# Verify data types after transformations
print(train_set['loan_grade'].head())
print(train_set['cb_person_default_on_file'].head())
print(train_set.info())  # To check if all object columns have been transformed

# Now apply the same steps to the test set
test_id = test_set['id']  # Store test IDs for final submission

test_set['cb_person_default_on_file'] = test_set['cb_person_default_on_file'].map({'Y': 1, 'N': 0}).astype(np.float32)

test_set['loan_grade_0'] = test_set['loan_grade']

test_set['loan_grade'] = test_set['loan_grade'].map({'A': 0.1, 'B': 0.2, 'C': 0.4, 'D': 0.5, 'E': 0.6, 'F': 0.8, 'G': 1}).astype(np.float32)

test_set = pd.get_dummies(test_set, columns=['loan_grade_0', 'person_home_ownership', 'loan_intent'])
test_set.drop(['id'], axis=1, inplace=True)

# Verify data types after transformations for the test set
print(test_set.info())


0    0.2
1    0.4
2    0.1
3    0.2
4    0.1
Name: loan_grade, dtype: float32
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: cb_person_default_on_file, dtype: float32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      58645 non-null  int64  
 1   person_income                   58645 non-null  int64  
 2   person_emp_length               58645 non-null  float64
 3   loan_grade                      58645 non-null  float32
 4   loan_amnt                       58645 non-null  int64  
 5   loan_int_rate                   58645 non-null  float64
 6   loan_percent_income             58645 non-null  float64
 7   cb_person_default_on_file       58645 non-null  float32
 8   cb_person_cred_hist_length      58645 non-null  int64  
 9   loan_status                     58645 non-nu

**Feature Engineering: Part2-create new features that might help increasing the accuracy**

In [71]:
# 1. Interaction between Income and Employment Length
train_set['income_emp_interaction'] = train_set['person_income'] * train_set['person_emp_length']

# 2. Interaction between Loan Amount and Interest Rate
train_set['loan_interest_interaction'] = train_set['loan_amnt'] * train_set['loan_int_rate']

# 3. Normalized Loan and Interest by Income
train_set['normalized_loan'] = train_set['loan_amnt'] / (train_set['person_income'] + 1)
train_set['normalized_interest'] = train_set['loan_int_rate'] / (train_set['person_income'] + 1)

# 4. Income Stability Score (income scaled by employment length)
train_set['income_stability_score'] = train_set['person_income'] / (train_set['person_emp_length'] + 1)

# 5. Age-Adjusted Income (income divided by age)
train_set['age_adjusted_income'] = train_set['person_income'] / (train_set['person_age'] + 1)

# 6. Log-Transformed Income and Loan Amount (reducing skewness)
train_set['log_income'] = np.log1p(train_set['person_income'])
train_set['log_loan_amnt'] = np.log1p(train_set['loan_amnt'])

# Repeat for test set:
test_set['income_emp_interaction'] = test_set['person_income'] * test_set['person_emp_length']
test_set['loan_interest_interaction'] = test_set['loan_amnt'] * test_set['loan_int_rate']
test_set['normalized_loan'] = test_set['loan_amnt'] / (test_set['person_income'] + 1)
test_set['normalized_interest'] = test_set['loan_int_rate'] / (test_set['person_income'] + 1)
test_set['income_stability_score'] = test_set['person_income'] / (test_set['person_emp_length'] +1)
test_set['age_adjusted_income'] = test_set['person_income'] / (test_set['person_age'] +1)
test_set['log_income'] = np.log1p(test_set['person_income'])
test_set['log_loan_amnt'] = np.log1p(test_set['loan_amnt'])

In [72]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd

# Separate features and target
y = train_set['loan_status']
X = train_set.drop(['loan_status'], axis=1)

# Initial split for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Initialize SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy=0.3, random_state=42)

# Oversample the training set
X_train, y_train = smote.fit_resample(X_train, y_train)

# Convert back to DataFrame to maintain column names
X_train = pd.DataFrame(X_train, columns=X.columns)
X_val = pd.DataFrame(X_val, columns=X.columns)

# Shuffle the training and validation sets after resampling
X_train, y_train = X_train.sample(frac=1, random_state=42).reset_index(drop=True), y_train.sample(frac=1, random_state=42).reset_index(drop=True)
X_val, y_val = X_val.sample(frac=1, random_state=42).reset_index(drop=True), y_val.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the count of True and False in both train and validation sets
print("Training set class distribution:\n", y_train.value_counts())
print("Validation set class distribution:\n", y_val.value_counts())


Training set class distribution:
 loan_status
0    45265
1    13579
Name: count, dtype: int64
Validation set class distribution:
 loan_status
0    5030
1     835
Name: count, dtype: int64


 **Predicting Using XGB**

In [74]:
# Define parameter search space
search_spaces = {
    'learning_rate': Real(0.001, 0.5, 'uniform'),
    'max_depth': Integer(2, 7),
    'n_estimators': Integer(100, 500),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'colsample_bytree': Real(0.5, 1.0, 'uniform'),
    'gamma': Real(0, 10, 'uniform'),
    'reg_alpha': Real(0, 10, 'uniform'),
    'reg_lambda': Real(0, 10, 'uniform')
}

# Initialize the XGBClassifier model with GPU support
model = xgb.XGBClassifier(
    objective='binary:logistic',  # Use for binary classification
    tree_method='hist',  # Use GPU for training
    verbosity=2
)

kf = KFold(n_splits=5)  # 10-fold cross-validation

# Using 'accuracy' as the scoring metric (can be changed to roc_auc, f1, etc.)
optimizer = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=32,
    cv=kf,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1,
    return_train_score=True
)

# Fit the optimizer using X_train and y_train (classification labels, not log-transformed)
optimizer.fit(X_train, y_train)

# Get the best model
xgb = optimizer.best_estimator_

# Get cross-validation results
cv_results = optimizer.cv_results_

# Print best parameters and mean accuracy
mean_accuracy = cv_results['mean_test_score']  # Mean accuracy for each fold

print(f"Best parameters: {optimizer.best_params_}")
print(f"Mean Cross-validation Accuracy: {mean_accuracy.mean():.4f}")
print(f"Cross-validation Accuracy for each iteration: {mean_accuracy}")

print("Optimization complete. Best XGBoost classifier model saved.")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [75]:
from sklearn.metrics import roc_auc_score, classification_report

y_pred_prob_XGB_train = xgb.predict_proba(X_train)
print(roc_auc_score(y_train, y_pred_prob_XGB_train[:, 1]))

y_pred_prob_XGB_val = xgb.predict_proba(X_val)
print(roc_auc_score(y_val, y_pred_prob_XGB_val[:, 1]))

0.9896649182477928
0.9551976762181402


 **Predicting using Logistic Regression**

In [76]:
from sklearn import linear_model

logr = linear_model.LogisticRegression(solver = 'saga', max_iter=20000)
logr.fit(X_train,y_train)

y_pred_prob_LR_train = logr.predict_log_proba(X_train)
print(roc_auc_score(y_train, y_pred_prob_LR_train[:, 1]))

y_pred_prob_LR_val = logr.predict_log_proba(X_val)
print(roc_auc_score(y_val, y_pred_prob_LR_val[:, 1]))

0.8329359112749447
0.8426342543541148


** Adding the predictions as new features**

In [79]:
# Make predictions using the best model directly on test_data
y_pred_prob_XGB_test = xgb.predict_proba(test_set)[:, 1]

# Prepare the output DataFrame
output = pd.DataFrame({'id': test_id.values, 'loan_status': y_pred_prob_XGB_test})

# Remove any duplicate rows by 'PassengerId'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('predictions.csv', index=False)
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>